# Related Papers
[1] Tao Luo#, Zhi-Qin John Xu #, Zheng Ma, Yaoyu Zhang*, Phase diagram for two-layer ReLU neural networks at infinite-width limit, arxiv 2007.07497 (2020), Journal of Machine Learning Research (2021) [pdf](https://ins.sjtu.edu.cn/people/xuzhiqin/pub/phasediagram2020.pdf), and in [arxiv](https://arxiv.org/abs/2007.07497). 

[2] Hanxu Zhou, Qixuan Zhou, Tao Luo, Yaoyu Zhang*, Zhi-Qin John Xu*, Towards Understanding the Condensation of Neural Networks at Initial Training. arxiv 2105.11686 (2021) [pdf](https://ins.sjtu.edu.cn/people/xuzhiqin/pub/initial2105.11686.pdf), and in [arxiv](https://arxiv.org/abs/2105.11686), see slides and [video talk in Chinese](https://www.bilibili.com/video/BV1tb4y1d7CZ/?spm_id_from%253D333.999.0.0), NeurIPS2022. 

For more details, see [xuzhiqin condense](https://ins.sjtu.edu.cn/people/xuzhiqin/pubcondense.html)



In [ ]:
import os
import time
import warnings
import numpy as np
import torch
import math
import torch.nn as nn
import torch.nn.functional as F
from typing import List
import argparse
import matplotlib
import matplotlib.pyplot as plt
import datetime
import re
import copy


# Condense 
![Condense](./pic/condense.png)
The above picture is the illstration of the condensation. 

#   Default configuration parameter settings.

In [ ]:
parser = argparse.ArgumentParser(description='PyTorch 1D dataset Training')



parser.add_argument('--lr', default=0.00001, type=float, help='learning rate')
parser.add_argument('--optimizer', default='adam',
                    help='optimizer: sgd | adam')
parser.add_argument('--epochs', default=2000, type=int,
                    metavar='N', help='number of total epochs to run')
parser.add_argument('--test_size',   default=10000, type=int,
                    help='the test size for model (default: 10000)')
parser.add_argument('--save', default='trained_nets',
                    help='path to save trained nets')
parser.add_argument('--save_epoch', default=10,
                    type=int, help='save every save_epochs')
parser.add_argument('--rand_seed', default=0, type=int,
                    help='seed for random num generator')
parser.add_argument('--t', type=float, default=2,
                    help='parameter initialization distribution variance power(We first assume that each layer is the same width.)')
parser.add_argument('--boundary', nargs='+', type=str, default=['-1', '1'],
                    help='the boundary of 1D data')
parser.add_argument('--training_size',   default=4, type=int,
                    help='the training size for model (default: 1000)')
parser.add_argument('--act_func_name', default='ReLU',
                    help='activation function')
parser.add_argument('--hidden_layers_width',
                    nargs='+', type=int, default=[1000])
parser.add_argument('--input_dim',   default=1, type=int,
                    help='the input dimension for model (default: 1)')
parser.add_argument('--output_dim',   default=1, type=int,
                    help='the output dimension for model (default: 1)')
parser.add_argument('--device',   default='cuda', type=str,
                    help='device used to train (cpu or cuda)')
parser.add_argument('--plot_epoch',   default=100, type=int,
                    help='step size of plotting interval (default: 1000)')
parser.add_argument('--ini_path', type=str,
                    default='')

args, unknown = parser.parse_known_args()


#   The storage path of the output file.

In [ ]:
def mkdirs(fn):  # Create directorys
    if not os.path.isdir(fn):
        os.makedirs(fn)
    return fn


def create_save_dir(path_ini): 
    subFolderName = re.sub(r'[^0-9]', '', str(datetime.datetime.now()))
    path = os.path.join(path_ini, subFolderName)
    mkdirs(path)
    mkdirs(os.path.join(path, 'output'))
    return path


args.path = create_save_dir(args.ini_path)


# Generation of training and test sets.
In this part, we use the target function $f(x)=0.2*ReLU(x-1/3) + 0.2*ReLU (-x-1/3)$

In [ ]:
def get_y(x):  # Function to fit
    # y = torch.sin(x*math.pi)
    y = 0.2*torch.relu(-1/3+x) + 0.2*torch.relu(-1/3-x)
    return y

for i in range(2):
    if isinstance(args.boundary[i], str):
        args.boundary[i] = eval(args.boundary[i])

args.test_input = torch.reshape(torch.linspace(args.boundary[0] - 0.5, args.boundary[1] + 0.5, steps=args.test_size), [args.test_size, 1])


args.training_input = torch.reshape(torch.linspace(args.boundary[0], args.boundary[1], steps=args.training_size), [args.training_size, 1])
args.test_target = get_y(args.test_input)
args.training_target = get_y(args.training_input)


def plot_target(args):

    plt.figure()
    ax = plt.gca()

    plt.plot(args.training_input.detach().cpu().numpy(),
             args.training_target.detach().cpu().numpy(), 'b*', label='True')
    plt.plot(args.test_input.detach().cpu().numpy(),
             args.test_target.detach().cpu().numpy(), 'r-', label='Test')

    ax.tick_params(labelsize=18)
    plt.legend(fontsize=18)
    plt.show()


plot_target(args)



In [ ]:
class xtanh(nn.Module):
    def __init__(self):
        super(xtanh,self).__init__()
    
    def forward(self,x):
        return x * nn.Tanh()(x) 
    
class x2tanh(nn.Module):
    def __init__(self):
        super(x2tanh,self).__init__()

    def forward(self,x):
        return x * x * nn.Tanh()(x)

# Network model and parameter initialization.

Given $\theta\in \mathbb{R}^M$, the FNN function $f_{\theta}(\cdot)$ is defined recursively. First, we denote $f^{[0]}_{\theta}(x)=x$ for all $x\in\mathbb{R}^d$. Then, for $l\in[L-1]$, $f^{[l]}_{\theta}$ is defined recursively as 
$f^{[l]}_{\theta}(x)=\sigma (W^{[l]} f^{[l-1]}_{\theta}(x)+b^{[l]})$, where $\sigma$ is a non-linear activation function.
Finally, we denote
\begin{equation*}
    f_{\theta}(x)=f(x,\theta)=f^{[L]}_{\theta}(x)=W^{[L]} f^{[L-1]}_{\theta}(x)+b^{[L]}.
\end{equation*}

The parameter initialization is under the Gaussian distribution as follows,

\begin{equation*}
    \theta|_{l} \sim N(0, \frac{1}{m_{l}^t}),
\end{equation*}
where the $l$ th layer parameters of $\theta$ is the ordered pair $\theta|_{l}=\Big(W^{[l]},b^{[l]}\Big),\quad l\in[L]$, $m_{l}$ is the width of the $l$ th layer.

In [ ]:
class Linear(nn.Module):
    def __init__(self, t, hidden_layers_width=[100],  input_size=20, num_classes: int = 1000, act_layer: nn.Module = nn.ReLU()):
        super(Linear, self).__init__()
        self.num_classes = num_classes
        self.input_size = input_size
        self.hidden_layers_width = hidden_layers_width
        self.t = t
        layers: List[nn.Module] = []
        self.layers_width = [self.input_size]+self.hidden_layers_width
        for i in range(len(self.layers_width)-1):
            layers += [nn.Linear(self.layers_width[i],
                                    self.layers_width[i+1]), act_layer]
        layers += [nn.Linear(self.layers_width[-1], num_classes, bias=False)]
        self.features = nn.Sequential(*layers)
        self._initialize_weights()


    def forward(self, x):

        x = x.view(x.size(0), -1)
        x = self.features(x)
        return x

    def _initialize_weights(self) -> None:

        for obj in self.modules():
            if isinstance(obj, (nn.Linear, nn.Conv2d)):
                nn.init.normal_(obj.weight.data, 0, 1 /
                                self.hidden_layers_width[0]**(self.t))
                if obj.bias is not None:
                    nn.init.normal_(obj.bias.data, 0, 1 /
                                    self.hidden_layers_width[0]**(self.t))


def get_act_func(act_func):
    if act_func == 'Tanh':
        return nn.Tanh()
    elif act_func == 'ReLU':
        return nn.ReLU()
    elif act_func == 'Sigmoid':
        return nn.Sigmoid()
    elif act_func == 'xTanh':
        return xtanh()
    elif act_func == 'x2Tanh':
        return x2tanh()
    else:
        raise NameError('No such act func!')


act_func = get_act_func(args.act_func_name)

model = Linear(args.t, args.hidden_layers_width, args.input_dim,
               args.output_dim, act_func).to(args.device)

para_init = copy.deepcopy(model.state_dict())


# One-step training function.

The training data set is denoted as  $S=\{(x_i,y_i)\}_{i=1}^n$, where $x_i\in\mathbb{R}^d$ and $y_i\in \mathbb{R}^{d'}$. For simplicity, we assume an unknown function $y$ satisfying $y(x_i)=y_i$ for $i\in[n]$. The empirical risk reads as
\begin{equation*}
    R_S(\theta)=\frac{1}{n}\sum_{i=1}^n\ell(f(x_i,\theta),y(x_i)),
\end{equation*}
where the loss function $\ell(\cdot,\cdot)$ is differentiable and the derivative of $\ell$ with respect to its first argument is denoted by $\nabla\ell(y,y^*)$. 

For a one-step gradient descent, we have, 

\begin{equation*}
    \theta_{t+1}=\theta_t-\eta\nabla R_S(\theta).
\end{equation*}

In [ ]:
def train_one_step(model, optimizer, loss_fn,  args):

    model.train()
    device = args.device
    data, target = args.training_input.to(
        device), args.training_target.to(device).to(torch.float)

    optimizer.zero_grad()
    outputs = model(data)
    loss = loss_fn(outputs, target)
    loss.backward()
    optimizer.step()

    return loss.item()


# One-step test function.

In [ ]:
def test(model, loss_fn, args):
    model.eval()
    device = args.device
    with torch.no_grad():
        data, target = args.test_input.to(
            device), args.test_target.to(device).to(torch.float)
        outputs = model(data)
        loss = loss_fn(outputs, target)

    return loss.item(), outputs


# Plot the loss value.

In [ ]:
def plot_loss(path, loss_train, x_log=False):

    plt.figure()
    ax = plt.gca()
    y2 = np.asarray(loss_train)
    plt.plot(y2, 'k-', label='Train')
    plt.xlabel('epoch', fontsize=18)
    ax.tick_params(labelsize=18)
    plt.yscale('log')
    if x_log == False:
        fntmp = os.path.join(path, 'loss.jpg')

    else:
        plt.xscale('log')
        fntmp = os.path.join(path, 'loss_log.jpg')
    plt.tight_layout()
    plt.savefig(fntmp,dpi=300)


    plt.close()


# Plot the output figure.


In [ ]:
def plot_model_output(path, args, output, epoch):

    plt.figure()
    ax = plt.gca()

    plt.plot(args.training_input.detach().cpu().numpy(),
             args.training_target.detach().cpu().numpy(), 'b*', label='True')
    plt.plot(args.test_input.detach().cpu().numpy(),
             output.detach().cpu().numpy(), 'r-', label='Test')

    ax.tick_params(labelsize=18)
    plt.legend(fontsize=18)
    fntmp = os.path.join(path, 'output', str(epoch)+'.jpg')

    plt.savefig(fntmp, dpi=300)


    plt.close()


# The expression of orientations and amplitudes of a Two-Layer Network

$$
f_{\boldsymbol{\theta}}(\boldsymbol{x})= \sum_{k=1}^m a_k \sigma\left(\boldsymbol{w}_k^{\top} \boldsymbol{x}\right)
$$

where $\boldsymbol{w}_k = (w_k,b_k)^T$ and $\boldsymbol{x} =(x,1)^T$.

The orientation of a neuron $\boldsymbol{w}_k$ is defined as $\frac{w_k}{\|\boldsymbol{w}_k\|_2}$. The amplitude of a neuron $\boldsymbol{w}_k$ is $|a_k|\cdot\|\boldsymbol{w_k}\|_2$

$a_k^0\sim \mathcal{N}(0,\beta_1^2),\quad \boldsymbol{w_k^0}\sim\mathcal{N}(0,\beta_2^2\boldsymbol{I}_d)$



\begin{equation*}
\gamma = \lim_{m\rightarrow \infty}-\frac{\log\beta_1\beta_2/\alpha}{\log m}, \quad \gamma'=\lim_{m\rightarrow \infty} -\frac{\log\beta_1/\beta_2}{\log m}
\end{equation*}

When $\sigma(x)=\mathrm{ReLU}(x)$, we have the following phase diagram.

![phase](./pic/phase.png)


In [ ]:
# To get the orientation and amplitude of each neuron in different layer.
def get_ori_A(checkpoint):

    wei1 = checkpoint['features.0.weight'].squeeze()
    bias = checkpoint['features.0.bias'].squeeze()
    wei2 = checkpoint['features.2.weight'].squeeze()
    wei = wei1 / (wei1 ** 2 + bias ** 2)**(1/2)

    bia = bias / (wei1 ** 2 + bias ** 2)**(1/2)
    ori = torch.sign(bia) * torch.acos(wei)
    A = wei2 * (wei1 ** 2 + bias ** 2)**(1/2)

    return ori, A

In [ ]:
# get the orientation and amplitude of each neuron at the beginning and end of training.
def get_ori_A_list(checkpoint_list):

    if not isinstance(checkpoint_list, list):
        ori, A=get_ori_A(checkpoint_list)
        return [ori], [A]

    else:
        ori_ini, A_ini = get_ori_A(checkpoint_list[0])

        ori, A = get_ori_A(checkpoint_list[1])

        return [ori_ini, ori], [A_ini, A]

In [ ]:
def plot_feature(path, ori, A, args, nota=''):
    """
    It plots the feature of the input data
    
    :param path: the path to save the figure
    :param ori: the feature orientation
    :param A: the feature amplitude
    :param args: the parameters of the model
    """

    if args.input_dim == 1:

        plt.figure()
        ax = plt.gca()

        if len(ori) == 1:

            ori = ori[0].squeeze().detach().cpu().numpy()
            A = A[0].squeeze().detach().cpu().numpy()

        elif len(ori) == 2:
            ori_ini, ori = ori[0].squeeze().detach().cpu(
            ).numpy(), ori[1].squeeze().detach().cpu().numpy()
            A_ini, A = A[0].squeeze().detach().cpu().numpy(
            ), A[1].squeeze().detach().cpu().numpy()

            print(ori_ini.shape, A_ini.shape)

            plt.scatter(ori_ini, abs(A_ini), color='cyan', label='ini feature')

        else:
            raise Exception('The length of the checkpoint list is less than or equal to two.')

        # mkdirs(r'%s\\feature' % (path))
        fn = mkdirs(os.path.join('%s'%path,'feature'))
        plt.scatter(ori, abs(A), color='r', label='fin feature')
        plt.xlim(-3.16,3.16)
        plt.xlabel('orientation', fontsize=18)
        plt.ylabel('amplitude', fontsize=18)
        plt.legend(fontsize=18)
        # fntmp = r'%s\\feature\\%s' % (path, nota)
        # fntmp = os.path.join(fn,'%s'%nota)
        # save_fig(plt, fntmp, pdf=False, x_log=False, y_log=True)
        plt.savefig(os.path.join(fn,'%s'%nota))
        plt.close()
    else:
        raise Exception('Input dimension must equal to 1!')

<!-- ![out05](./999.jpg)![out1](./3999.jpg)![out2](./1999.jpg) -->
<center class="half">
    <img src="./pic/999.jpg" width="300"/><img src="./pic/3999.jpg" width="300"/><img src="./pic/1999.jpg" width="300"/>
</center>

<center class="half">
    <img src="./pic/999.png" width="300"/><img src="./pic/3999.png" width="300"/><img src="./pic/1999.png" width="300"/>
</center>

The above picture shows three typical case: linear(left), critical(middle), condense(right)


# Main.

In [ ]:
# optimizer = torch.optim.SGD(model.parameters(), lr=args.lr)
args.t = 1
args.lr = 0.05
args.epochs = 10000
args.save_epoch = 1000
args.plot_epoch = 1000
args.optimizer = 'sgd'
args.act_func_name = 'ReLU'
# args.path = create_save_dir(args.ini_path)
args.savepath = create_save_dir(os.path.join(args.path, 't=%s'%args.t))
act_func = get_act_func(args.act_func_name)

model = Linear(args.t, args.hidden_layers_width, args.input_dim,
               args.output_dim, act_func).to(args.device)

para_init = copy.deepcopy(model.state_dict())
if args.optimizer=='sgd':
  optimizer = torch.optim.SGD(model.parameters(), lr=args.lr)
else:
  optimizer = torch.optim.Adam(model.parameters(), lr=args.lr)
loss_fn = nn.MSELoss(reduction='mean')
t0 = time.time()
loss_training_lst=[]
loss_test_lst = []
for epoch in range(args.epochs+1):

      model.train()
      loss = train_one_step(
        model, optimizer, loss_fn, args)
      loss_test, output = test(
          model, loss_fn, args)
      loss_training_lst.append(loss)
      loss_test_lst.append(loss_test)
      if epoch % args.plot_epoch == 0:
            print("[%d] loss: %.6f valloss: %.6f time: %.2f s" %
                  (epoch + 1, loss, loss_test, (time.time()-t0)))
  
      if (epoch+1) % (args.plot_epoch) == 0:
          plot_loss(path=args.savepath,
                    loss_train=loss_training_lst, x_log=True)
          plot_loss(path=args.savepath,
                    loss_train=loss_training_lst, x_log=False)

          
          para_now = copy.deepcopy(model.state_dict())
          ori, A = get_ori_A_list([para_init,para_now])
          plot_feature(args.savepath, ori, A, args,nota='%s'%epoch)

          plot_model_output(args.savepath, args, output, epoch)


# Multiplicity of the activation function
Suppose that $\sigma(x)$ satisfies the following condition, there exists a $p\in \mathbb{N}$ and $p\geq 1$, such that the $s$-th order derivatives $\sigma^{(s)}(x)(0)=0$ for $s=1,2,\cdots,p-1$ and $\sigma^{(p)}(0) \neq 0$, then we say $\sigma(x)$ has the multiplicity $p$. 

* $tanh(x)$ has the multiplicity $1$.
* $xtanh(x)$ has the multiplicity $2$.


In [ ]:
# optimizer = torch.optim.SGD(model.parameters(), lr=args.lr)
args.t = 2
args.lr = 0.000001
args.epochs = 500
args.save_epoch = 100
args.plot_epoch = 10
args.optimizer = 'adam'
args.act_func_name = 'Tanh'
args.savepath = create_save_dir(os.path.join(args.path, 'initial%s'%args.act_func_name))

act_func = get_act_func(args.act_func_name)

model = Linear(args.t, args.hidden_layers_width, args.input_dim,
               args.output_dim, act_func).to(args.device)

para_init = copy.deepcopy(model.state_dict())
if args.optimizer=='sgd':
  optimizer = torch.optim.SGD(model.parameters(), lr=args.lr)
else:
  optimizer = torch.optim.Adam(model.parameters(), lr=args.lr)
loss_fn = nn.MSELoss(reduction='mean')
t0 = time.time()
loss_training_lst=[]
loss_test_lst = []
for epoch in range(args.epochs+1):

      model.train()
      loss = train_one_step(
        model, optimizer, loss_fn, args)
      loss_test, output = test(
          model, loss_fn, args)
      loss_training_lst.append(loss)
      loss_test_lst.append(loss_test)
      if epoch % args.plot_epoch == 0:
            print("[%d] loss: %.6f valloss: %.6f time: %.2f s" %
                  (epoch + 1, loss, loss_test, (time.time()-t0)))
  
      if (epoch+1) % (args.plot_epoch) == 0:
          plot_loss(path=args.savepath,
                    loss_train=loss_training_lst, x_log=True)
          plot_loss(path=args.savepath,
                    loss_train=loss_training_lst, x_log=False)

          
          para_now = copy.deepcopy(model.state_dict())
          ori, A = get_ori_A_list([para_init,para_now])
          plot_feature(args.savepath, ori, A, args,nota='%s'%epoch)

          plot_model_output(args.savepath, args, output, epoch)


The above code is about initial condense in different activation functions with different multiplicity.

# Task: 
try to find the relation between the number of the directions in initial condense and the multiplicity.